In [1]:
import numpy as np
import torch

# inits (Konstruktor-Aufrufe) der Boundary Conditions in verschiedenen house-Skripten von M.Kliemank

## NEX
Der Aufruf mit "self.units", der in Arbeit und Paper (?) verwendet wurde, sollte korrekt sein. Dabei wird u auch als Profil übergeben (initial solution -> u -> BC)

In [3]:
# house3D, single
lt.NonEquilibriumExtrapolationInletU(self.units.lattice,np.array([self.units.characteristic_velocity_lu, 0, 0]), [0, 0, 1])

# SIMULATION_CODE.py (ARBEIT), house3D_final, house3D_final_new2_noverhang, kommt_so_in_die_arbeit, newMPI, proper_single, PU_Umrechnung, TestGeometry, Verification_Dist_House
lt.NonEquilibriumExtrapolationInletU(self.units.lattice, self.units, [-1, 0, 0], np.array(u))


## EQ_inlet
Als Backup-Inlet kann ich die EQ_PU mit korrektem (!) Geschwindigkeitsprofil nutzen, wenn ich z.B. die IBB implementiere und es crasht, um das vergleichen zu können.
Möglicherweise könnte ich die Initialisierung noch an meine Cylinder-Flows anpassen...

In [3]:
# house3D_final, house3D_final_new2_noverhang, proper_multi, proper_single, PU_Umrechnung
lt.EquilibriumBoundaryPU(np.abs(x) < 1e-6, self.units.lattice, self.units, u[:, 0, ...], p[0, 0, ...])



## ZG_outlet
Solange ich hier keine Crashes o.ä. beobachte, sollte ich das so verwenden können.
Da über diese Boundary auch keine großen Gradienten bzw. Flüsse stattfinden (zeige mal u_z), sollte da auch kein großes Stabilitätsproblem bestehen 

In [4]:
# house3D_final, house3D_final_new2_noverhang, kommt_so_in_die_arbeit, 3D_literature, house3D_literature_neueBoundary, newMPI, proper_multi, proper_single, PU_Umrechnung, TestGeometry, TestSEMBoundary, Verification_Dist_House
lt.ZeroGradientOutlet(self.units.lattice, [0, 0, 1])

# EQ_P_outlet
Bis auf den vermutlich neuesten Code (h3D_lit._neueBoundary) wird hier jeweils nicht die Dichte übergeben, d.h. diese dafaultet zu rho0=1.0
Frage: wie weit weicht Druck_PU=0 als Dichte von Dichte_LU=1 ab? => im Testcase (24.5.24) kommt da auch 1.0 raus, d.h. das macht keinen Unterschied, ich kanns aber drin lassen, falls das später mal einen Unterschied macht!

In [ ]:
# SIMULATION_CODE.py (ARBEIT), house3D_final, house3D_final_new2_noverhang, kommt_so_in_die_arbeit, 3D_literature, newMPI, proper_multi, proper_single, PU_Umrechnung, TestGeometry, Verification_Dist_House
lt.EquilibriumOutletP(self.units.lattice, [1, 0, 0])

# house3D_literature_neueBoundary, TestSEMBoundary (NEUEST?)
lt.EquilibriumOutletP(self.units.lattice, [1, 0, 0], rho0=self.units.convert_pressure_pu_to_density_lu(0))

# SEI (synthetic eddy inlet)
Außer der Parametrisierungsmöglichkeit (übergabe von Variablen anstelle hardcoded), ist das nichts anders im Init.

In [ ]:
# 3D_literature
lt.SyntheticEddyInlet(self.units.lattice, self.units, self.rgrid, rho=self.units.convert_density_to_pu(self.units.convert_pressure_pu_to_density_lu(0)), u_0=self.units.characteristic_velocity_pu, K=10, L=0.75, N=150, R=self.reyolds_stress_tensor, velocityProfile=self.wind_speed_profile)

# house3D_literature_neueBoundary, TestSEMBoundary
lt.SyntheticEddyInlet(self.units.lattice, self.units, self.rgrid, rho=self.units.convert_density_to_pu(self.units.convert_pressure_pu_to_density_lu(0)), u_0=self.units.characteristic_velocity_pu, K=self.K_Factor*10, L=L, N=N, R=self.reyolds_stress_tensor, velocityProfile=self.wind_speed_profile)

In [ ]:
## für SEI könnte auch die Belegung von K, L, N etc. in "boundaries(self):" relevant sein...

# TestSEMHouseEmpty, TestSEMBoundary
x, y, z = self.grid
L = self.L
if self.N == 0:
    N = int(np.ceil((self.units.convert_length_to_pu(self.resolution_y) * self.units.convert_length_to_pu(self.resolution_z)) / (4 * L**2)))
else:
    N = self.N
    
# house3D_literature -> da wird es im init komplett "händisch" gefült (s.o.)